<a href="https://colab.research.google.com/github/dashakudjaeva/NLP_facult/blob/dashakudjaeva-HW1/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
import pandas as pd
!pip install pymorphy2 > None
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import numpy as np
from string import punctuation
from nltk.tokenize import word_tokenize

In [141]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [142]:
import nltk
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language='english')

In [143]:
from statistics import mean

In [144]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [145]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS

In [146]:
from sklearn.model_selection import train_test_split

In [147]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [148]:
import re

In [149]:
from sklearn.feature_extraction.text import TfidfTransformer,  TfidfVectorizer
from sklearn.pipeline import Pipeline

In [150]:
from sklearn.naive_bayes import MultinomialNB

In [151]:
import eli5

In [152]:
from sklearn.tree import DecisionTreeClassifier

In [173]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

1. Загрузить набор данных Spam or Not Spam

In [153]:
db = pd.read_csv('spam.csv', encoding='iso-8859-1')[['v1', 'v2']].rename(columns={'v1': 'label', 'v2': 'text'})
db

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


2. Предобработка текстов

In [154]:
db['prep'] = np.nan

In [155]:
nlp = spacy.load("en_core_web_sm")
stopwords = nlp.Defaults.stop_words
print(f'Spacy english stopwords size: {len(stopwords)}', end='\n\n')
' '.join(stopwords)

Spacy english stopwords size: 326



"some neither thence toward on various nowhere take which others last must however re as well moreover more by everywhere during then n‘t yourself made down seemed after for doing you upon ‘s our something if too them mostly than somehow everyone several being otherwise put whence serious while ours only was wherever could your another an over each throughout became twenty give towards yourselves there 'm else really thus other is therein side how from anyhow top whatever go perhaps where one until his except or had may used its meanwhile am get within 're what herein ‘re bottom when next yours whenever fifteen often formerly beforehand just be did three becomes ‘ll per amount ’s hers first anyone not herself front whither alone that up see always ‘m myself thereafter along whether behind nothing sometime much ever their should becoming whereby already done therefore most hence against elsewhere cannot least none keep ‘ve both because here regarding quite whereas itself seeming do whol

In [156]:
db['prep'] = db['text'].apply(
    lambda x: ' '.join(
        token.lemma_.lower() for token in nlp(x) if
        not token.is_stop
        and not token.is_punct
        and not token.is_digit
        and not token.like_email
        and not token.like_num
        and not token.is_space
    )
)
db.head()

,label,text,prep
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts tex...
3,ham,U dun say so early hor... U c already then say...,u dun early hor u c
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think go usf live


CountVectorizer

In [157]:
X_train, X_test, y_train, y_test = train_test_split(db['prep'], db['label'], random_state=2023)

In [158]:
vectorizer = CountVectorizer(max_df=0.7, min_df=0.003) # max_df фильтрует corpus-specific stop words
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [159]:
def custom_tokenize(text):
  text = re.sub(r'[^a-zA-Z ]', '', text)
  return text.split()

In [160]:
custom_vectorizer = CountVectorizer(
    max_df=0.7,
    min_df=0.003,
    tokenizer=custom_tokenize, # аналогично можно переопределить preprocessor
)
X_train_vectorized = custom_vectorizer.fit_transform(X_train)
X_test_vectorized = custom_vectorizer.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [161]:
X_train_vectorized

<4179x445 sparse matrix of type '<class 'numpy.int64'>'
	with 18708 stored elements in Compressed Sparse Row format>

In [162]:
m=pd.DataFrame(X_train_vectorized.toarray(), columns=custom_vectorizer.get_feature_names_out())
m

,able,abt,account,actually,add,address,aft,afternoon,age,ah,...,xmas,xxx,y,ya,yeah,year,yes,yesterday,yo,yup
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4174,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4176,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4177,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [163]:
logreg = LogisticRegression().fit(X_train_vectorized, y_train)
preds = logreg.predict(X_test_vectorized)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         ham       0.97      1.00      0.99      1207
        spam       0.99      0.83      0.90       186

    accuracy                           0.98      1393
   macro avg       0.98      0.92      0.95      1393
weighted avg       0.98      0.98      0.98      1393



TfidfVectorizer

In [164]:
tf_transformer = TfidfTransformer(use_idf=False).fit(m)
word_freq = tf_transformer.transform(m)
tf= pd.DataFrame(word_freq.todense(), columns=custom_vectorizer.get_feature_names_out())
tf

,able,abt,account,actually,add,address,aft,afternoon,age,ah,...,xmas,xxx,y,ya,yeah,year,yes,yesterday,yo,yup
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4174,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4175,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4176,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4177,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [165]:
pipe = Pipeline(
    steps=[
        ('tfidf', TfidfVectorizer()),
        ('clf', LogisticRegression())
    ]
).fit(X_train, y_train)
preds = pipe.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1207
        spam       0.99      0.70      0.82       186

    accuracy                           0.96      1393
   macro avg       0.97      0.85      0.90      1393
weighted avg       0.96      0.96      0.96      1393



Обучение моделей

DecisionTreeClassifier

In [166]:
pipe1 = Pipeline(
    steps=[
        ('tfidf', TfidfVectorizer()),
        ('clf', DecisionTreeClassifier())
    ]
).fit(X_train, y_train)
preds1 = pipe1.predict(X_test)
print(classification_report(y_test, preds1))

              precision    recall  f1-score   support

         ham       0.98      0.99      0.98      1207
        spam       0.90      0.87      0.89       186

    accuracy                           0.97      1393
   macro avg       0.94      0.93      0.93      1393
weighted avg       0.97      0.97      0.97      1393



In [167]:
parameter_grid1 = {
    "tfidf__norm": ("l1", "l2"),
    "clf__splitter": ("random", "best"),
}

In [174]:
grid_search1 = HalvingGridSearchCV(
    pipe1,
    param_grid=parameter_grid1,
    n_jobs=-1,
    verbose=1,
    cv=2,
    scoring='accuracy',
    random_state=42,
)
grid_search1.fit(X_train, y_train)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 1393
max_resources_: 4179
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 4
n_resources: 1393
Fitting 2 folds for each of 4 candidates, totalling 8 fits
----------
iter: 1
n_candidates: 2
n_resources: 4179
Fitting 2 folds for each of 2 candidates, totalling 4 fits


HalvingGridSearchCV(cv=2,
                    estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               DecisionTreeClassifier())]),
                    n_jobs=-1,
                    param_grid={'clf__splitter': ('random', 'best'),
                                'tfidf__norm': ('l1', 'l2')},
                    random_state=42, scoring='accuracy', verbose=1)

In [175]:
preds1_1 = grid_search1.best_estimator_.predict(X_test)
print(classification_report(y_test, preds1_1))

              precision    recall  f1-score   support

         ham       0.98      0.99      0.98      1207
        spam       0.92      0.86      0.89       186

    accuracy                           0.97      1393
   macro avg       0.95      0.92      0.94      1393
weighted avg       0.97      0.97      0.97      1393



LogisticRegression

In [176]:
pipe2 = Pipeline(
    steps=[
        ('tfidf', TfidfVectorizer()),
        ('clf', LogisticRegression())
    ]
).fit(X_train, y_train)
preds2 = pipe2.predict(X_test)
print(classification_report(y_test, preds2))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1207
        spam       0.99      0.70      0.82       186

    accuracy                           0.96      1393
   macro avg       0.97      0.85      0.90      1393
weighted avg       0.96      0.96      0.96      1393



In [177]:
parameter_grid2 = {
    "tfidf__norm": ("l1", "l2"),
    "clf__C": (0, 100),
}

In [178]:
grid_search2 = HalvingGridSearchCV(
    pipe2,
    param_grid=parameter_grid2,
    n_jobs=-1,
    verbose=1,
    cv=2,
    scoring='accuracy',
    random_state=42,
)
grid_search2.fit(X_train, y_train)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 1393
max_resources_: 4179
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 4
n_resources: 1393
Fitting 2 folds for each of 4 candidates, totalling 8 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
4 fits failed out of a total of 8.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1160, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-pa

----------
iter: 1
n_candidates: 2
n_resources: 4179
Fitting 2 folds for each of 2 candidates, totalling 4 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.96623563 0.97126437 0.9736778  0.98037774]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the train scores are non-finite: [       nan        nan 1.         1.         0.99856402 1.        ]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


HalvingGridSearchCV(cv=2,
                    estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf', LogisticRegression())]),
                    n_jobs=-1,
                    param_grid={'clf__C': (0, 100),
                                'tfidf__norm': ('l1', 'l2')},
                    random_state=42, scoring='accuracy', verbose=1)

In [179]:
preds2_1 = grid_search2.best_estimator_.predict(X_test)
print(classification_report(y_test, preds2_1))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1207
        spam       0.99      0.87      0.93       186

    accuracy                           0.98      1393
   macro avg       0.98      0.93      0.96      1393
weighted avg       0.98      0.98      0.98      1393



Multinomial Naive Bayes

In [180]:
pipe3 = Pipeline(
    steps=[
        ('tfidf', TfidfVectorizer()),
        ('clf', MultinomialNB())
    ]
).fit(X_train, y_train)
preds3 = pipe2.predict(X_test)
print(classification_report(y_test, preds3))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1207
        spam       0.99      0.70      0.82       186

    accuracy                           0.96      1393
   macro avg       0.97      0.85      0.90      1393
weighted avg       0.96      0.96      0.96      1393



In [181]:
parameter_grid3 = {
    "tfidf__norm": ("l1", "l2"),
    "clf__alpha": (0, 10),
}

In [182]:
grid_search3 = HalvingGridSearchCV(
    pipe3,
    param_grid=parameter_grid3,
    n_jobs=-1,
    verbose=1,
    cv=2,
    scoring='accuracy',
    random_state=42,
)
grid_search3.fit(X_train, y_train)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 1393
max_resources_: 4179
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 4
n_resources: 1393
Fitting 2 folds for each of 4 candidates, totalling 8 fits
----------
iter: 1
n_candidates: 2
n_resources: 4179
Fitting 2 folds for each of 2 candidates, totalling 4 fits


/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:629: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:635: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


HalvingGridSearchCV(cv=2,
                    estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf', MultinomialNB())]),
                    n_jobs=-1,
                    param_grid={'clf__alpha': (0, 10),
                                'tfidf__norm': ('l1', 'l2')},
                    random_state=42, scoring='accuracy', verbose=1)

In [183]:
preds3_1 = grid_search3.best_estimator_.predict(X_test)
print(classification_report(y_test, preds3_1))

              precision    recall  f1-score   support

         ham       0.98      0.99      0.98      1207
        spam       0.91      0.87      0.89       186

    accuracy                           0.97      1393
   macro avg       0.94      0.93      0.94      1393
weighted avg       0.97      0.97      0.97      1393



Сравнение качества обученных моделей

Изучив, что показывают метрики из отчёта, я пришла к выводу, что значение f1-score наиболее важно при оценке работы метода, так как оно объединяет в себе показатели точности и отзыва и выдаёт средневзвешенное гармоническое значение. Что касается вопроса о том, какое значение по строчке нужно рассматривать у f1-score, я считаю, что так как нам важны все сентименты и нам нужно оценить всю работу метода, нужно взять значение f1-score по строчке weighted avg. Сравним методы по этому показателю: лучшей оказалась Логистическая регрессия при подобранных гиперпараметрах.